# Lesson_3

1. Обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. Вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
3. Вывести сравнение полученных моделей по метрикам бизнеса по показателям с урока
    - стоимость лечения 15000р, если сделали тест и начали лечить вовремя
    - стоимость лечения 20000р, если упустили и начали лечить когда уже проявились все симптомы
    - стоимость теста 1400р
4. Сделать выводы о том, какая модель справилась с задачей лучше других
5. *Найти порог классификации по деньгам для лучшей модели
    - Стоимость лечения 15000р, если сделали тест и начали лечить вовремя
    - Стоимость лечения 20000р, если упустили и начали лечить когда уже проявились все симптомы
    - Стоимость теста 1400р

## 1).

In [2]:
import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score, precision_recall_curve, roc_curve, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('train_case2.csv', ';')
df.sample(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
25547,36457,22536,1,150,79.0,140,90,3,3,0,0,1,1
18702,26708,17610,2,154,78.0,120,80,1,1,0,0,1,0
19240,27489,16728,1,164,59.0,110,70,1,1,0,0,1,1


In [4]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='cardio'),
                                                   df['cardio'], random_state=2)

In [5]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, key):
        self.key = key
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.key]
    

class NumberSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, key):
        self.key = key
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[[self.key]]
    

class OHEEncoder(BaseEstimator, TransformerMixin):
    
    def __init__(self, key):
        self.key = key
        
    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self
    
    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]
    
from sklearn.preprocessing import StandardScaler

continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transformer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transformer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

In [6]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

In [7]:
feature_processing.fit_transform(X_train)

array([[ 0.63862212,  0.68719022, -0.15189705, ...,  1.        ,
         0.        ,  1.        ],
       [-0.80462358, -1.50641457, -0.8459871 , ...,  0.        ,
         0.        ,  1.        ],
       [-0.20452226, -0.04401137,  1.51391908, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 1.59131847,  0.32158943,  1.23628306, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.11277269, -0.28774524, -0.8459871 , ...,  0.        ,
         0.        ,  0.        ],
       [-0.52952642,  0.07785556,  1.02805604, ...,  0.        ,
         0.        ,  1.        ]])

#### LogisticRegression

In [8]:
from sklearn.linear_model import LogisticRegression


classifier_log = Pipeline([
    ('features', feats),
    ('classifier', LogisticRegression(random_state=15))
])

# запустим кросс-валидацию

cv_scores = cross_val_score(classifier_log, X_train, y_train, cv=7, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print(f'CV score in ({cv_score} +- {cv_score_std})')

classifier_log.fit(X_train, y_train)
y_score_log = classifier_log.predict_proba(X_test)[:, 1]

CV score in (0.7874680986554055 +- 0.0026869871307792378)


In [9]:
# Посчитаем метрики
b = 1
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score_log)
fscore = (1 + b**2) * (precision * recall) / (b**2 * precision + recall)
# максимальное значение f-score
ix = np.argmax(fscore)
precision_log = precision[ix]
recall_log = recall[ix]
fscore_log = fscore[ix]
thresholds_log = thresholds[ix]
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]}, Precision={precision[ix]}, Recall={recall[ix]}')

Best Threshold=0.40548000050388566, F-Score=0.7332441058375373, Precision=0.6664795058955643, Recall=0.8148741418764303


In [11]:
print(f'roc auc score: {roc_auc_score(y_train, classifier_log.predict_proba(X_train)[:, 1])}')

roc_auc_log = roc_auc_score(y_test, classifier_log.predict_proba(X_test)[:, 1])
print(f'roc auc score: {roc_auc_log}')

roc auc score: 0.787908978992452
roc auc score: 0.783400585666071


#### RandomForest

In [12]:
from sklearn.ensemble import RandomForestClassifier

In [13]:
%%time
classifier_random_forest = Pipeline([
    ('features', feats),
    ('classifier', RandomForestClassifier(random_state=15, n_estimators=15, min_samples_leaf=15))
])

# запустим кросс-валидацию

cv_scores = cross_val_score(classifier_random_forest, X_train, y_train, cv=7, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print(f'CV score in ({cv_score} +- {cv_score_std})')

classifier_random_forest.fit(X_train, y_train)
y_score_random_forest = classifier_random_forest.predict_proba(X_test)[:, 1]

CV score in (0.800120053053112 +- 0.0024200585693701106)
CPU times: total: 3min 9s
Wall time: 3min 14s


In [36]:
# Посчитаем метрики
b = 1
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score_random_forest)
fscore = (1 + b**2) * (precision * recall) / (b**2 * precision + recall)
# максимальное значение f-score
ix = np.nanargmax(fscore)
precision_forest = precision[ix]
recall_forest = recall[ix]
fscore_forest = fscore[ix]
thresholds_forest = thresholds[ix]
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]}, Precision={precision[ix]}, Recall={recall[ix]}')

Best Threshold=0.36749514090106394, F-Score=0.7415232402349788, Precision=0.6745734108381773, Recall=0.8232265446224256


In [37]:
print(f'roc auc score: {roc_auc_score(y_train, classifier_random_forest.predict_proba(X_train)[:, 1])}')

roc_auc_forest = roc_auc_score(y_test, classifier_random_forest.predict_proba(X_test)[:, 1])
print(f'roc auc score: {roc_auc_forest}')

roc auc score: 0.8354144639784491
roc auc score: 0.7965363546597286


#### Catboost

In [51]:
from catboost import CatBoostClassifier

In [17]:
%%time
classifier_catboost = Pipeline([
    ('features', feats),
    ('classifier', CatBoostClassifier(random_state=15,
                                      silent=True))
])

# запустим кросс-валидацию

cv_scores = cross_val_score(classifier_catboost, X_train, y_train, cv=7, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print(f'CV score in ({cv_score} +- {cv_score_std})')

classifier_catboost.fit(X_train, y_train)
y_score_catboost = classifier_catboost.predict_proba(X_test)[:, 1]

CV score in (0.8025428751396235 +- 0.003032884487355725)
CPU times: total: 10min 16s
Wall time: 2min 23s


In [38]:
# Посчитаем метрики
b = 1
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score_catboost)
fscore = (1 + b**2) * (precision * recall) / (b**2 * precision + recall)
# максимальное значение f-score
ix = np.nanargmax(fscore)
precision_catb = precision[ix]
recall_catb = recall[ix]
fscore_catb = fscore[ix]
thresholds_catb = thresholds[ix]
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]}, Precision={precision[ix]}, Recall={recall[ix]}')

Best Threshold=0.3547559604856096, F-Score=0.744485294117647, Precision=0.6722611582441903, Recall=0.834096109839817


In [19]:
print(f'roc auc score: {roc_auc_score(y_train, classifier_catboost.predict_proba(X_train)[:, 1])}')

roc_auc_catb = roc_auc_score(y_test, classifier_catboost.predict_proba(X_test)[:, 1])
print(f'roc auc score: {roc_auc_catb}')

roc auc score: 0.8342554622329773
roc auc score: 0.7998760749401794


#### LightGBM

In [20]:
from lightgbm import LGBMClassifier as lgbm

In [21]:
%%time
classifier_lgbm = Pipeline([
    ('features', feats),
    ('classifier', lgbm(random_state=15))
])

# запустим кросс-валидацию

cv_scores = cross_val_score(classifier_lgbm, X_train, y_train, cv=7, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print(f'CV score in ({cv_score} +- {cv_score_std})')

classifier_lgbm.fit(X_train, y_train)
y_score_lgbm = classifier_lgbm.predict_proba(X_test)[:, 1]

CV score in (0.8034548599611272 +- 0.0022798028047318897)
CPU times: total: 23.3 s
Wall time: 3.16 s


In [22]:
# Посчитаем метрики
b = 1
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score_lgbm)
fscore = (1 + b**2) * (precision * recall) / (b**2 * precision + recall)
# максимальное значение f-score
ix = np.nanargmax(fscore)
precision_lgbm = precision[ix]
recall_lgbm = recall[ix]
fscore_lgbm = fscore[ix]
thresholds_lgbm = thresholds[ix]
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]}, Precision={precision[ix]}, Recall={recall[ix]}')

Best Threshold=0.3656438335871373, F-Score=0.7430394730070506, Precision=0.6752408567954354, Recall=0.8259725400457666


In [23]:
print(f'roc auc score: {roc_auc_score(y_train, classifier_lgbm.predict_proba(X_train)[:, 1])}')

roc_auc_lgbm = roc_auc_score(y_test, classifier_lgbm.predict_proba(X_test)[:, 1])
print(f'roc auc score: {roc_auc_lgbm}')

roc auc score: 0.8256878990051495
roc auc score: 0.8000824490872804


## 2).

In [40]:
pd.DataFrame({
    'model': ['logreg', 'random_forest', 'catboost', 'lightgbm'],
    'precision': [precision_log, precision_forest, precision_catb, precision_lgbm],
    'recall': [recall_log, recall_forest, recall_catb, recall_lgbm],
    'fscore': [fscore_log, fscore_forest, fscore_catb, fscore_lgbm],
    'roc_auc': [roc_auc_log, roc_auc_forest, roc_auc_catb, roc_auc_lgbm]
})

,model,precision,recall,fscore,roc_auc
0,logreg,0.666480,0.814874,0.733244,0.783401
1,random_forest,0.674573,0.823227,0.741523,0.796536
2,catboost,0.672261,0.834096,0.744485,0.799876
3,lightgbm,0.675241,0.825973,0.743039,0.800082


## 3).

In [41]:
rubl_test=1400
rubl_early_treatment=15000
rubl_late_treatment=20000

In [42]:
def busines_lost(y_test, y_score, threshold):
    cnf_matrix = confusion_matrix(y_test, y_score > threshold)
    TN = cnf_matrix[0,0]
    TP = cnf_matrix[1,1]
    FN = cnf_matrix[1,0]
    FP = cnf_matrix[0,1]
    lost = (TP + FP) * rubl_test + FN * rubl_late_treatment + TP * rubl_early_treatment
    
    return lost / 10**6

In [43]:
losts = pd.DataFrame({'model': ['logreg', 'random_forest', 'catboost', 'lightgbm'],
    'busines_losts': [
        busines_lost(y_test, y_score_log, thresholds_log),
        busines_lost(y_test, y_score_random_forest, thresholds_forest),
        busines_lost(y_test, y_score_catboost, thresholds_catb),
        busines_lost(y_test, y_score_lgbm, thresholds_lgbm)
    ]
})
losts.sort_values(by='busines_losts')

,model,busines_losts
2,catboost,153.5352
3,lightgbm,153.6760
1,random_forest,153.7610
0,logreg,154.1540


## 4).

Лучше всего себя показывают модели градиентного бустинга Catboost и Lightgbm.
Из-за несущественных отличий между lgbm и catb лучше выбирать lgbm, так как затраты мощностей и время создания модели гораздо меньше, а это существенно ускорит работу при подборе гиперпараметров модели.

## *5).

In [44]:
def business_threshold_calibrate(y_test, y_score, thresholds,
                                 rubl_test=100, rubl_early_treatment=100, rubl_late_treatment=1000):
    business_threshold = 0
    r_test_all_r_ML_ = 0

    rubl_ML_ = 1_000_000_000 

    rs = []
    n = 100
    for opt_buisness_tr in np.linspace(0, 1, n).tolist():
        # подберем порог для улучшения бизнесс показателя

        cnf_matrix = confusion_matrix(y_test, y_score > (opt_buisness_tr))
        TN = cnf_matrix[0][0]
        FN = cnf_matrix[1][0]
        TP = cnf_matrix[1][1]
        FP = cnf_matrix[0][1]

        rubl_ML = (TP + FP) * rubl_test + FN * rubl_late_treatment + TP * rubl_early_treatment
        
        if rubl_ML < rubl_ML_:            
            business_threshold = opt_buisness_tr
            
            rubl_ML_ = rubl_ML
            
        rs.append(rubl_ML)   

    return rubl_ML_ / 10**6

In [45]:
busines_tr_log = business_threshold_calibrate(y_test, y_score_log, 
                                          thresholds_log,
                                          rubl_test=1400, 
                                          rubl_early_treatment=15000, 
                                          rubl_late_treatment=20000)

In [46]:
busines_tr_forest = business_threshold_calibrate(y_test, y_score_random_forest, 
                                          thresholds_forest,
                                          rubl_test=1400, 
                                          rubl_early_treatment=15000, 
                                          rubl_late_treatment=20000)

In [47]:
busines_tr_catb = business_threshold_calibrate(y_test, y_score_catboost, 
                                          thresholds_catb,
                                          rubl_test=1400, 
                                          rubl_early_treatment=15000, 
                                          rubl_late_treatment=20000)

In [48]:
busines_tr_lgbm = business_threshold_calibrate(y_test, y_score_lgbm, 
                                          thresholds_lgbm,
                                          rubl_test=1400, 
                                          rubl_early_treatment=15000, 
                                          rubl_late_treatment=20000)

In [49]:
losts = pd.DataFrame({'model': ['logreg', 'random_forest', 'catboost', 'lightgbm'],
    'busines_losts': [busines_tr_log, busines_tr_forest,
                      busines_tr_catb, busines_tr_lgbm]
})
losts.sort_values(by='busines_losts')

,model,busines_losts
3,lightgbm,153.2484
2,catboost,153.2950
1,random_forest,153.3438
0,logreg,153.6580


In [50]:
print(f'Если всем делать тесты: {(rubl_test * y_test.shape[0] + rubl_early_treatment * y_test.value_counts()[1]) / 10**6} млн. рублей')
print(f'Если никому не делать тесты: {(rubl_late_treatment * y_test.value_counts()[1]) / 10**6} млн. рублей')

Если всем делать тесты: 155.6 млн. рублей
Если никому не делать тесты: 174.8 млн. рублей


**В среднем разница между решением модели с отсечкой по бизнес метрике и решением сделать тесты всем около 2 млн. рублей только на тестовой выборке - 25% от общей.**